In [4]:
# ============================================================
# XGBoost + Feature Engineering + Ridge Stacking (OOF feature)
# For Kaggle "Predicting Student Test Scores" (PS S6E1)
#
# - Feature engineering:
#   * high_study
#   * study_att (interaction)
#   * manual_formula (LUT-based linear score)
#   * sin/cos transforms for study_hours + class_attendance (periods 12, 14)
#   * a few safe numeric transforms (sqrt, square, ratios)
#
# - Stacking:
#   * Ridge OOF predictions on engineered features -> feature_lr_pred
#
# - Model:
#   * XGBoost GPU with early stopping inside folds
#   * Final fit on full training using avg best_iteration
#
# Output:
#   * prints CV RMSE
#   * writes submission.csv (id, exam_score) in DATA_DIR
# ============================================================

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor

DATA_DIR = "/rds/rds-lxu/ml_datasets/exam_score_predict"
TRAIN_PATH = f"{DATA_DIR}/train.csv"
TEST_PATH  = f"{DATA_DIR}/test.csv"

ID_COL = "id"
TARGET = "exam_score"

# -----------------------------
# 1) Load
# -----------------------------
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

y = train_df[TARGET].astype(float).values
train_X0 = train_df.drop(columns=[TARGET]).copy()
test_X0  = test_df.copy()

print("train:", train_df.shape, "test:", test_df.shape)
print("Initial train shape:", train_X0.shape, "test shape:", test_X0.shape)

# -----------------------------
# 2) Feature engineering
# -----------------------------
LUT = {
    "sleep_quality": {"good": 5, "average": 0, "poor": -5},
    "facility_rating": {"high": 4, "medium": 0, "low": -4},
    "study_method": {"coaching": 10, "mixed": 5, "group study": 2, "online videos": 1, "self-study": 0},
}

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # ensure consistent string normalization for LUT mapping
    for c in ["sleep_quality", "facility_rating", "study_method", "gender", "course", "internet_access", "exam_difficulty"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    # core engineered features
    df["high_study"] = (df["study_hours"] >= 7.0).astype(int)
    df["study_att"] = df["study_hours"] * df["class_attendance"]

    # safe numeric transforms
    df["study_hours_sq"] = df["study_hours"] ** 2
    df["class_attendance_sq"] = df["class_attendance"] ** 2
    df["sleep_hours_sq"] = df["sleep_hours"] ** 2

    df["study_hours_sqrt"] = np.sqrt(np.clip(df["study_hours"], 0.0, None))
    df["sleep_hours_sqrt"] = np.sqrt(np.clip(df["sleep_hours"], 0.0, None))

    df["att_per_hour"] = df["class_attendance"] / (df["study_hours"] + 1e-3)
    df["study_per_sleep"] = df["study_hours"] / (df["sleep_hours"] + 1e-3)

    # manual formula (as described in discussions)
    sq = df["sleep_quality"].map(LUT["sleep_quality"]).fillna(0.0)
    sm = df["study_method"].map(LUT["study_method"]).fillna(0.0)
    fr = df["facility_rating"].map(LUT["facility_rating"]).fillna(0.0)

    df["manual_formula"] = (
        6.0 * df["study_hours"]
        + 0.35 * df["class_attendance"]
        + 1.5 * df["sleep_hours"]
        + sq + sm + fr
    )

    # sinusoidal / cosine features (nonlinear transforms)
    for p in [12, 14]:
        df[f"study_hours_sin_{p}"] = np.sin(2 * np.pi * df["study_hours"] / p)
        df[f"study_hours_cos_{p}"] = np.cos(2 * np.pi * df["study_hours"] / p)
        df[f"class_attendance_sin_{p}"] = np.sin(2 * np.pi * df["class_attendance"] / p)
        df[f"class_attendance_cos_{p}"] = np.cos(2 * np.pi * df["class_attendance"] / p)

    return df

train_X = add_features(train_X0)
test_X  = add_features(test_X0)

# -----------------------------
# 3) One-hot (train+test aligned)
# -----------------------------
# keep IDs separately
train_id = train_X[ID_COL].values
test_id  = test_X[ID_COL].values

# combine then one-hot on object columns
all_X = pd.concat([train_X, test_X], axis=0, ignore_index=True)

obj_cols = all_X.select_dtypes(include=["object"]).columns
all_X[obj_cols] = all_X[obj_cols].astype(str).fillna("missing")
all_X = all_X.fillna(0.0)

all_X_enc = pd.get_dummies(all_X, columns=obj_cols, drop_first=False)

X_train = all_X_enc.iloc[:len(train_X)].values
X_test  = all_X_enc.iloc[len(train_X):].values

print("Encoded feature dim:", X_train.shape[1])

# -----------------------------
# 4) Ridge stacking feature: OOF predictions -> feature_lr_pred
# -----------------------------
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# RidgeCV picks alpha via inner CV; fast + strong for linear signal
ridge = RidgeCV(alphas=np.logspace(-4, 4, 25), fit_intercept=True)

oof_ridge = np.zeros(len(X_train), dtype=float)
test_ridge_folds = np.zeros((N_SPLITS, len(X_test)), dtype=float)

for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train), 1):
    X_tr, y_tr = X_train[tr_idx], y[tr_idx]
    X_va, y_va = X_train[va_idx], y[va_idx]

    ridge.fit(X_tr, y_tr)
    oof_ridge[va_idx] = ridge.predict(X_va)
    test_ridge_folds[fold - 1] = ridge.predict(X_test)

test_ridge = test_ridge_folds.mean(axis=0)

# append stacking feature
X_train_stack = np.hstack([X_train, oof_ridge.reshape(-1, 1)])
X_test_stack  = np.hstack([X_test,  test_ridge.reshape(-1, 1)])

print("After stacking dim:", X_train_stack.shape[1])

# -----------------------------
# 5) XGBoost CV with early stopping
# -----------------------------
xgb_params = dict(
    n_estimators=20000,           # large cap; early stop chooses effective trees
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=3.0,
    subsample=0.85,
    colsample_bytree=0.85,
    reg_lambda=2.0,
    reg_alpha=0.0,
    gamma=0.0,
    objective="reg:squarederror",
    eval_metric="rmse",
    tree_method="hist",
    device="cuda:0",
    early_stopping_rounds=200,    # must have eval_set per fold
    verbosity=0,
)

oof_xgb = np.zeros(len(X_train_stack), dtype=float)
test_pred_folds = np.zeros((N_SPLITS, len(X_test_stack)), dtype=float)
best_iters = []

for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_stack), 1):
    X_tr, y_tr = X_train_stack[tr_idx], y[tr_idx]
    X_va, y_va = X_train_stack[va_idx], y[va_idx]

    model = XGBRegressor(**xgb_params, random_state=42 + fold)
    model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=200)

    oof_xgb[va_idx] = model.predict(X_va)
    test_pred_folds[fold - 1] = model.predict(X_test_stack)

    best_iters.append(int(model.best_iteration) + 1)
    rmse_fold = float(np.sqrt(mean_squared_error(y_va, oof_xgb[va_idx])))
    print(f"[Fold {fold}] best_iter={best_iters[-1]}  RMSE={rmse_fold:.6f}")

rmse_oof = float(np.sqrt(mean_squared_error(y, oof_xgb)))
print("\n=== XGB CV Summary ===")
print(f"OOF RMSE: {rmse_oof:.6f}")
print("Best iters:", best_iters)
best_n = int(np.mean(best_iters))
print("Final n_estimators =", best_n)

# -----------------------------
# 6) Final fit on full train (no early stopping) + predict test
# -----------------------------
final_params = xgb_params.copy()
final_params.pop("early_stopping_rounds", None)
final_params["n_estimators"] = best_n
final_params["verbosity"] = 1

final_model = XGBRegressor(**final_params, random_state=123)
final_model.fit(X_train_stack, y, verbose=False)

test_pred = final_model.predict(X_test_stack)

# clip to plausible range (optional, but often helps a bit)
test_pred = np.clip(test_pred, 0.0, 100.0)

# -----------------------------
# 7) Submission
# -----------------------------
submission = pd.DataFrame({ID_COL: test_id, TARGET: test_pred})
out_path = f"{DATA_DIR}/submission.csv"
submission.to_csv(out_path, index=False)

print("\nWrote:", out_path)
print(submission.head())
print("Submission columns:", list(submission.columns))


train: (630000, 13) test: (270000, 12)
Initial train shape: (630000, 12) test shape: (270000, 12)
Encoded feature dim: 49
After stacking dim: 50
[0]	validation_0-rmse:18.41953
[200]	validation_0-rmse:8.77020
[400]	validation_0-rmse:8.72996
[600]	validation_0-rmse:8.70791
[800]	validation_0-rmse:8.69438
[1000]	validation_0-rmse:8.68662
[1200]	validation_0-rmse:8.68089
[1400]	validation_0-rmse:8.67752
[1600]	validation_0-rmse:8.67517
[1800]	validation_0-rmse:8.67272
[2000]	validation_0-rmse:8.67116
[2200]	validation_0-rmse:8.66995
[2400]	validation_0-rmse:8.66999
[2472]	validation_0-rmse:8.67037
[Fold 1] best_iter=2273  RMSE=8.669538
[0]	validation_0-rmse:18.48868
[200]	validation_0-rmse:8.77458
[400]	validation_0-rmse:8.73446
[600]	validation_0-rmse:8.71117
[800]	validation_0-rmse:8.69851
[1000]	validation_0-rmse:8.68972
[1200]	validation_0-rmse:8.68409
[1400]	validation_0-rmse:8.68000
[1600]	validation_0-rmse:8.67716
[1800]	validation_0-rmse:8.67483
[2000]	validation_0-rmse:8.67219
[22